In [1]:
# !python3 -m spacy download en_core_web_sm

In [1]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import re
import time
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
def get_ent(result_out,doc_txt):
    doc = nlp(doc_txt)
    ORG=[]
    Dates=[]
    Money=[]
    Other=[]
    for ent in doc.ents:
        if ent.label_.upper() in ['PERSON','ORG']:
            ORG.append(ent.text)
        elif ent.label_.upper() in ['TIME','DATE']:
            Dates.append(ent.text)
        elif ent.label_.upper() in ['MONEY']:
            Money.append(ent.text)
        else:
            Other.append(ent.text)

    add_to_dict(result_out,'ORG','|'.join(ORG))
    add_to_dict(result_out,'Money','|'.join(Money))
    add_to_dict(result_out,'Other','|'.join(Other))
    add_to_dict(result_out,'Dates','|'.join(Dates))

In [3]:
result_out={}

In [4]:
# doc = nlp(" Lupin contributes Rs 21 crores towards COVID-19 relief efforts EP News Bureau Apr 11, 2020 The contributions were made separately by Lupin’s employees, the Company and the promoter family")

In [5]:
def add_to_dict(dictionary, key, value):
    if type(value)==list:
        for val in value:
            setKey(dictionary,key,val)
    else:
        if key not in dictionary:
            dictionary[key] = [value]
        elif type(dictionary[key]) == list:
            dictionary[key].append(value)
        else:
            dictionary[key] = [dictionary[key], value]

def get_filtered_article(article_list,keyword_list,result_out,page_no):
    for article in article_list:
        art_txt=article.text
        art_txt_clean=re.sub("[^A-Za-z0-9 ]", " ",art_txt)
        art_txt_clean=re.sub(" +", " ",art_txt_clean).lower()
        match_found=bool(re.search(keyword_list,art_txt_clean,re.IGNORECASE))
        art_url_title=article.find_element_by_tag_name('a')
        attr_url=''
        attr_url=art_url_title.get_attribute('href')
        # print(attr_url)
        # attr_title=art_url_title.text
        if match_found==True:
            match_words=re.findall(keyword_list,art_txt_clean,re.IGNORECASE)
            # add_to_dict(result_out,'Article_Title',attr_title)
            add_to_dict(result_out,'Article_Text',art_txt)
            add_to_dict(result_out,'Web_Page_No',page_no)
            add_to_dict(result_out,'Article_URL',attr_url)
            add_to_dict(result_out,'Matchd_words','|'.join(match_words))
            get_ent(result_out,art_txt)
        


In [6]:
# !pip install webdriver-manager
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(ChromeDriverManager().install())

In [7]:
# os.chdir("C:\\Users\\DINES\\Desktop\\Development\\Amod_DataScience\\WebScrap\\TenderApp\\")
driver = webdriver.Chrome('/home/optimus/.wdm/drivers/chromedriver/81.0.4044.69/linux64/chromedriver')
driver.maximize_window()


## Keyword List

In [8]:
keyword_list=(' contribut| aid| donate| fund')
article_list={}

In [9]:
# result_out={}
# pageNo=1
# for page_index in range(1,pageNo+1):
#     driver.get('https://grantstation.com/covid-19-related-funding')
#     article_list=driver.find_elements_by_class_name('views-row')
#     get_filtered_article(article_list,keyword_list,result_out,page_index)
# df_out=pd.DataFrame.from_dict(result_out)
# df_out.to_csv('Output/bbc.csv')

In [10]:
# result_out={}
# df_out=pd.DataFrame()
# pageNo=1

# driver.get('https://grantstation.com/covid-19-related-funding')
# next_btn=driver.find_element_by_class_name('pager__item--next').find_element_by_tag_name('a')
# for page_index in range(0,pageNo+1):
#     driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
#     time.sleep(2)
#     next_btn.click()
# time.sleep(3)
# article_list=driver.find_elements_by_class_name('views-row')
# get_filtered_article(article_list,keyword_list,result_out,page_index)
# df_out=pd.DataFrame.from_dict(result_out)
# df_out.to_csv('Output/covid.csv')

In [9]:
page=driver.get('https://www.infoedglobal.com/COVID19/COVID-19.html')
ele_tbl=driver.find_element_by_tag_name('body')
tbl_HTML=ele_tbl.get_attribute("outerHTML")
soup=BeautifulSoup(tbl_HTML,'html.parser')
df_out=pd.read_html(tbl_HTML,header=0)
df_result=pd.DataFrame()
for df in df_out:
    df_result=df_result.append(df)
df_result.to_csv('Output/infoedg.csv')

In [ ]:
driver.get('https://www.infoedglobal.com/COVID19/COVID-19.html')
next_btn=driver.find_element_by_class_name("k-link k-pager-nav k-pager-last").find_element_by_tag_name('span')
for page_index in range(1,38):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(2)
    next_btn.click()


In [ ]:
from selenium.webdriver.support.ui import Select
driver.get('https://www.infoedglobal.com/COVID19/COVID-19.html')
select = Select(driver.find_element_by_xpath("//select[@value='all']"))
all_selected_options = select.all_selected_options
options = select.options
driver.find_element_by_id("all").click()

In [9]:
result_out={}
pageNo=5
df_result=pd.DataFrame()
driver.get('https://projects.worldbank.org/en/projects-operations/projects-list?searchTerm=covid-19')
for page_index in range(1,pageNo):
    time.sleep(2)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    next_ele=driver.find_elements_by_class_name('ng-tns-c1-0')[page_index]
    print(next_ele)
    next_ele.click()
    time.sleep(2)
    ele_tbl=driver.find_element_by_tag_name('body')
    tbl_HTML=ele_tbl.get_attribute("outerHTML")
    soup=BeautifulSoup(tbl_HTML,'html.parser')
    df_out=pd.read_html(tbl_HTML,header=0)
    for df in df_out:
        df_result=df_result.append(df)

<selenium.webdriver.remote.webelement.WebElement (session="96a93cba005a176daabc43aa54925dae", element="b7e9364e-7adf-428f-a367-360ad46ea586")>


/home/optimus/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


<selenium.webdriver.remote.webelement.WebElement (session="96a93cba005a176daabc43aa54925dae", element="1636e7b5-abd4-4d51-a27a-eadf02366522")>
<selenium.webdriver.remote.webelement.WebElement (session="96a93cba005a176daabc43aa54925dae", element="32f3df93-ab8f-4898-b311-9a6859d9307c")>
<selenium.webdriver.remote.webelement.WebElement (session="96a93cba005a176daabc43aa54925dae", element="aed2f523-a991-4c6b-8814-4214dfe4d42e")>


In [10]:
df_result

,Approval Date,Commitment Amount,Country,End Date,Project ID,Project Title,Start Date,Status,Unnamed: 2
0,"April 24, 2020",16.95,Chad,NaN,P173894,Chad COVID-19 Strategic Preparedness and Respo...,NaN,Active,NaN
1,"April 24, 2020",100.00,Turkey,NaN,P173988,Turkey Emergency COVID-19 Health Project,NaN,Active,NaN
2,"April 23, 2020",36.20,Bosnia and Herzegovina,NaN,P173809,Bosnia and Herzegovina Emergency COVID-19 Project,NaN,Active,NaN
3,"April 22, 2020",100.00,Philippines,NaN,P173877,Philippines COVID-19 Emergency Response Project,NaN,Active,NaN
4,"April 21, 2020",2.90,Samoa,NaN,P173920,Samoa COVID-19 Emergency Response Project,NaN,Active,NaN
...,...,...,...,...,...,...,...,...,...
15,"April 10, 2020",25.80,Mali,NaN,P173816,MALI COVID-19 EMERGENCY RESPONSE PROJECT,NaN,Active,NaN
16,"April 10, 2020",20.00,Papua New Guinea,NaN,P173834,Papua New Guinea COVID-19 Emergency Response P...,NaN,Active,NaN
17,"April 10, 2020",5.00,Burundi,NaN,P173845,Burundi COVID19 Preparedness and Response Project,NaN,Active,NaN
18,"April 9, 2020",7.50,Liberia,NaN,P173812,Liberia COVID-19 Emergency Response Project,NaN,Active,NaN
